In [2]:
# !pip install mecab-python3

# !pip install unidic-lite
# !pip install japanize-matplotlib

In [1]:
import MeCab
import unidic
import sys
import re
import unicodedata
import matplotlib.pyplot as plt
import tqdm
import japanize_matplotlib
import os
import collections
import glob
import pandas as pd

In [2]:
# Check if MeCab works
# t = MeCab.Tagger('-r/nul -d"C:/Program Files (x86)/MeCab/dic/ipadic"')

t = MeCab.Tagger('-r/nul -d"C:/Program Files (x86)/MeCab/dic/ipadic"')

sentence = 'すもももももももものうち'
print(t.parse(sentence))

すもも	名詞,一般,*,*,*,*,すもも,スモモ,スモモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
うち	名詞,非自立,副詞可能,*,*,*,うち,ウチ,ウチ
EOS



### Here is a translation 
- すもも	名詞 (= noun),一般 (=general),*,*,*,*,すもも,スモモ,スモモ
- も	助詞 (=particle) ,係助詞 (=binding particle),*,*,*,*,も,モ,モ
- もも	名詞 (= noun),一般 (=general),*,*,*,*,もも,モモ,モモ
- も	助詞 (=particle) ,係助詞 (=binding particle),*,*,*,*,も,モ,モ
- もも	名詞 (= noun),一般 (=general),*,*,*,*,もも,モモ,モモ
- の	助詞 (=particle) ,連体化(=attributive),*,*,*,*,の,ノ,ノ
- うち	名詞(=noun),非自立(=non-indipendent),副詞可能(adverb)=,*,*,*,うち,ウチ,ウチ

In [3]:
#create the list of meaningless words such as "that" "year" "what" etc... 
with open("stopwords.txt", encoding='UTF-8') as f:
    stopwords = f.read().split("\n")
print(stopwords)

['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ', 'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか', 'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら', 'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', 'ごっちゃ', 'これ', 'これら', 'ごろ', 'さまざま', 'さらい', 'さん', 'しかた', 'しよう', 'すか', 'ずつ', 'すね', 'すべて', 'ぜんぶ', 'そう', 'そこ', 'そちら', 'そっち', 'そで', 'それ', 'それぞれ', 'それなり', 'たくさん', 'たち', 'たび', 'ため', 'だめ', 'ちゃ', 'てん', 'とおり', 'とき', 'どこ', 'どこか', 'ところ', 'どちら', 'どっか', 'どっち', 'どれ', 'なか', 'なかば', 'なに', 'など', 'なん', 'はじめ', 'はず', 'はるか', 'ひと', 'ひとつ', 'ふく', 'ぶり', 'べつ', 'へん', 'ぺん', 'ほう', 'ほか', 'まさ', 'まし', 'まとも', 'まま', 'みたい', 'みつ', 'みなさん', 'みんな', 'もと', 'もの', 'もん', 'やつ', 'よう', 'よそ', 'わけ', 'わたし', '', 'ハイ', '', '', '上', '中', '下', '字', '', '', '年', '月', '日', '時', '分', '秒', '週', '火', '水', '木', '金', '土', '国', '都', '道', '府', '県', '市', '区', '町', '村', '', '', '各', '第', '方', '何', '的', '度', '文', '者', '体', '人', '他', '今', '部', '課', '係', '外', '類', '達', '気', '室', '口', '誰', '用', '界', '会', '首', '別', '話', '私', '屋', '店', '家', '場', '等', '見', '際', '観', 

In [4]:

def count_word(text, pos_tags, stopwords):
    
    words_list = []
    tagger = MeCab.Tagger('-r/nul -d"C:/Program Files (x86)/MeCab/dic/ipadic"')
    
    node = tagger.parseToNode(text)

    while node:
        features = node.feature.split(",")
        base = features[6]
        pos_tag = features[0]
        
        # remove "stopwords"
        if pos_tag in pos_tags and base not in stopwords:
            words_list.append((base, pos_tag))
            

        node = node.next
    count = collections.Counter(words_list)

    return count

# chart
# def draw_barh_chart(y, width):
#     fig, ax = plt.subplots(figsize=(8, 4.5))
#     ax.barh(y, width)

# #     fig.savefig("word_count.png")


# if __name__ == "__main__":
#     # read files
#     #change HERE to try other files
#     with open('F:\Wikipedia\wikiextractor-master\wikiextractor\people\people鈴木みのり (カーリング選手)', 'r', encoding='UTF-8') as files:
#         text = files.read()
    
#     # read stopwords
#     with open("stopwords.txt",encoding='UTF-8') as f:
#         stopwords = f.read().split("\n")

#     pos_tags = ["名詞", "動詞", "形容詞"] 
#     #  ["名詞", "動詞", "形容詞"]=  ["noun", "verb", "adj"]
#     count = count_word(text, pos_tags, stopwords)

# top100_words = count.most_common(100)
# top100_words
    
# top100 = pd.DataFrame(top100_words, columns = ['word_tuple', "count"]) 
# top100
   
    
    
# Draw a chart
#top_words = count.most_common(10)
#words = list(map(lambda x: x[0], top_words))[::-1]
#counts = list(map(lambda x: x[1], top_words))[::-1]
#draw_barh_chart(words, counts)

In [5]:
file_list = glob.glob("F:/Wikipedia/wikiextractor-master/wikiextractor/maleFigureSkater/*") 

In [6]:
file_list[:5]

['F:/Wikipedia/wikiextractor-master/wikiextractor/maleFigureSkater\\イルハン・マンスズ',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/maleFigureSkater\\本田武史',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/maleFigureSkater\\髙橋大輔 (フィギュアスケート選手)',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/maleFigureSkater\\アレクセイ・ヤグディン',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/maleFigureSkater\\田村岳斗']

In [7]:
# read stopwords
with open("stopwords.txt",encoding='UTF-8') as f:
    stopwords = f.read().split("\n")

pos_tags = ["名詞", "動詞", "形容詞"] 
#  ["名詞", "動詞", "形容詞"]=  ["noun", "verb", "adj"]

In [8]:
dfs = []

In [9]:
for f in file_list:
    with open(f, 'r', encoding='UTF-8') as file:
        text = file.read()
    
    f_name = f.split("\\")[-1]
    
    count = count_word(text, pos_tags, stopwords)
    
    #top100_words = count.most_common(100)
    all_words = count.items()
    
    all_words_df = (
        pd
        .DataFrame(all_words, columns = ['word_tuple', "count"])
        .assign(
            word = lambda x: x["word_tuple"].apply(lambda wt: wt[0]),
            pos = lambda x: x["word_tuple"].apply(lambda wt: wt[1]),
            file_name = f_name
        )
        .sort_values("count", ascending=False)
    )
    #CHANGE PATH!!!!!!!
    full_csv_file = "F:/Wikipedia/wikiextractor-master/wikiextractor/maleFigureSkater/new_csvs/" + f_name + "_words.csv"
    #print(f_name)
    all_words_df.to_csv(full_csv_file, index=None)
    dfs.append(all_words_df)


PermissionError: [Errno 13] Permission denied: 'F:/Wikipedia/wikiextractor-master/wikiextractor/maleFigureSkater\\csv'

In [10]:
overall = (
    pd
    .concat(dfs)
    .groupby(["word", "pos"])
    .pipe(
        lambda grp: pd.DataFrame({
            "total": grp["count"].sum(),
            "articles": grp["file_name"].nunique()
        }))
    .reset_index()
    .sort_values("total", ascending=False)
)

In [11]:
overall[overall.pos == "名詞"].head(60)

,word,pos,total,articles
6038,選手権,名詞,2454,458
2104,位,名詞,2414,291
923,スケート,名詞,2123,776
876,シーズン,名詞,1515,222
1321,フィギュア,名詞,1485,775
6037,選手,名詞,1279,775
2859,回転,名詞,1124,93
1874,世界,名詞,1081,289
2367,出場,名詞,999,226
2238,優勝,名詞,943,311


In [12]:
overall[overall.pos == "動詞"].head(60)

,word,pos,total,articles
3098,始める,動詞,240,170
4242,果たす,動詞,221,107
2267,入る,動詞,193,93
5145,終わる,動詞,172,86
5194,続く,動詞,167,50
2683,受ける,動詞,166,73
4449,決める,動詞,134,50
5484,行う,動詞,127,60
254,できる,動詞,110,52
4755,生まれる,動詞,104,100


In [13]:
overall[overall.pos == "形容詞"].head(60)

,word,pos,total,articles
269,ない,形容詞,100,54
6369,高い,形容詞,80,31
2997,大きい,形容詞,45,29
2981,多い,形容詞,33,21
4633,無い,形容詞,29,20
5890,近い,形容詞,19,16
3486,強い,形容詞,17,9
3994,新しい,形容詞,17,10
5402,良い,形容詞,14,11
6244,難しい,形容詞,13,11


In [14]:
overall.to_csv("maleFigureSkater_text_overall.csv", index= False)

In [17]:
df= pd.read_csv("maleFigureSkater_text_overall.csv")
df

,word,pos,total,articles
0,選手権,名詞,2454,458
1,位,名詞,2414,291
2,スケート,名詞,2123,776
3,シーズン,名詞,1515,222
4,フィギュア,名詞,1485,775
...,...,...,...,...
6426,域,名詞,1,1
6427,執念,名詞,1,1
6428,基,名詞,1,1
6429,基づく,動詞,1,1
